In [1]:
from helpers.db_helpers import SqliteHelper
from helpers.settings import DEMO_DB_PATH
import pandas as pd

from dotenv import load_dotenv
import mindsdb_sdk

import os

# copy .env.example and rename to .env and add your writer api key and org id
load_dotenv()


True

In [2]:
#set mindsdb project name

project_name = 'writer_demo'

In [3]:
# Connect to local server
server = mindsdb_sdk.connect()
server = mindsdb_sdk.connect('http://127.0.0.1:47334')


In [4]:
#create project

server.create_project(project_name)

RuntimeError: Project with name 'writer_demo' already exists

In [ ]:
# project
project = server.get_project(project_name)

print(project)

In [ ]:
#create local sqlite

data = [
	{
	"table_name": "context",
	"data": "../data/qa/drug_context_cleaned.csv"
},
	{
	"table_name": "qa",
	"data": "../data/qa/questions.csv"
}
]

sqlite_helper = SqliteHelper(db_name="../data/qa/qa.db")

sqlite_helper.create_tables(
	data=data
)

In [ ]:
#add sqlite demo db to mindsdb

server.drop_database("qa_writer")
server.create_database("qa_writer",engine='sqlite',connection_args={'db_file':DEMO_DB_PATH})

In [ ]:
#create writer engine
query = project.query('CREATE ML_ENGINE writer FROM writer')
query.fetch()

In [42]:
#drop model
print(project)
project.drop_model("writer_qa_demo")

Project(writer_demo)


In [43]:
#get context column names

context_columns = pd.read_csv("../data/qa/drug_context_cleaned.csv").columns.tolist()

#ensure correct project
project = server.get_project(project_name)
print(project)

#set Mindsdb model name
model_name='writer_qa_demo'

#create model
writer_api_key = os.getenv("WRITER_API_KEY")
writer_org_id = os.getenv("WRITER_ORG_ID")

options=dict(
	model_name='palmyra-x',
	prompt_template='{question}',
	writer_org_id=writer_org_id,
	writer_api_key=writer_api_key,
	context_columns=context_columns,
)

model = project.create_model(
	name=model_name,
	predict="type_of_product",
	database="qa_writer",
	query="select * from context limit 3",
	engine="writer",
	options=options
)


Project(writer_demo)


In [63]:
#run predictions

df =project.query("SELECT * FROM writer_qa_demo WHERE question='How to treat a sore throat?'").fetch()


In [13]:
# model_name='writer_qa_demo'
# project = server.get_project(project_name)
# model = project.get_model(model_name)
# df2 = model.predict(pred_df)

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://127.0.0.1:47334/api/projects/writer_demo/models/writer_qa_demo/predict

In [64]:
df.columns

Index(['question', 'answer', 'source_documents'], dtype='object')

In [65]:
print(df.head(5))

                      question  \
0  How to treat a sore throat?   

                                              answer  \
0  {"choices":[{"text":"The correct answer is: se...   

                                    source_documents  
0  {'sources_content': ['medical help right away ...  


In [67]:
df['source_documents'][0]

{'sources_content': ['medical help right away Sore throat warning if sore throat is severe persists for more than 2 days is accompanied or followed by fever headache rash nausea or vomiting consult a doctor promptly',
  'help right away Sore throat warning if sore throat is severe persists more than 2 days is accompanied or followed by fever headache rash nausea or vomitting consult a doctor promptly',
  'seek medical help right away Sore throat warning If sore throat is severe persists for more than 2 days is accompanied or followed by fever headache rash nausea or vomiting consult a doctor promptly Do not use DAYTIME NIGHTTIME with any other drug containing acetaminophen prescription or nonprescription If you are not sure whether a drug contains acetaminophen ask a doctor or pharmacist if you are now taking a prescription monoamine oxidase inhibitor MAOI certain drugs for depression psychiatric',
  'drug take other drugs containing prescription or nonprescription NSAIDs aspirin ibupr